# MNIST classification in Keras

Author: Michał Słapek

Digit recognition as classification example.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dense, Dropout, Flatten

In [ ]:
num_classes = 10

In [ ]:
from keras.datasets import mnist

(X_train_valid, y_train_valid), (X_test, y_test) = mnist.load_data()

In [ ]:
y_train_valid.shape

In [ ]:
X_train_valid.shape

In [ ]:
y_test.shape

In [ ]:
img = X_train_valid[10]

print(img.shape)
print(img)

In [ ]:
plt.imshow(img)

## Multilayer perceptron

Based on https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py

In [ ]:
batch_size = 128
epochs = 20

In [ ]:
MLP_X_train_valid = X_train_valid.reshape(60000, 28 * 28)
MLP_X_train_valid = MLP_X_train_valid.astype('float32')
MLP_X_train_valid /= 255

from sklearn.utils import shuffle
MLP_X_train_valid, MLP_y_train_valid = shuffle(MLP_X_train_valid, y_train_valid, random_state=356549)

In [ ]:
MLP_X_train_valid[10]

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(28 * 28,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)
model.summary()

In [ ]:
model.load_weights('weights/mnist_mlp.hdf5')
# history = model.fit(
#     MLP_X_train_valid, MLP_y_train_valid,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=1/3)
# model.save_weights('weights/mnist_mlp.hdf5')

In [ ]:
model.evaluate(MLP_X_train_valid[-20_000:], MLP_y_train_valid[-20_000:])

## Convolutional Neural Network

Based on https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

In [ ]:
CNN_X_train_valid = X_train_valid
CNN_X_train_valid = CNN_X_train_valid[:, np.newaxis, :, :]
CNN_X_train_valid = CNN_X_train_valid.astype('float32')
CNN_X_train_valid /= 255

from sklearn.utils import shuffle
CNN_X_train_valid, CNN_y_train_valid = shuffle(CNN_X_train_valid, y_train_valid, random_state=356549)

In [ ]:
print(CNN_X_train_valid[10].shape)
plt.imshow(CNN_X_train_valid[10][0])
plt.colorbar()
print(CNN_y_train_valid[10])

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(1, 28, 28),
                 data_format='channels_first'))
model.add(Conv2D(64, (3, 3), activation='relu', data_format='channels_first'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)
model.summary()

In [ ]:
model.load_weights('weights/mnist_cnn.hdf5')
# history = model.fit(
#     CNN_X_train_valid, CNN_y_train_valid,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=1/3)
# model.save_weights('weights/mnist_cnn.hdf5')

In [ ]:
model.evaluate(CNN_X_train_valid[-20_000:], CNN_y_train_valid[-20_000:])

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

images = CNN_X_train_valid[[-5, -320, -434, -500, 9733-20_000, 15104-20_000, 15866-20_000]]
predictions = model.predict(images)

for i in range(len(images)):
    plt.imshow(images[i][0])
    probs = predictions[i]
    plt.title(f'Predicted {predictions[i]}, best {np.argmax(probs)}')
    plt.show()